# Storytelling Data Visualization Lab

In this lab you'll use a dataset called `housing_prices.csv` which contains the sales data of houses. The dataset and descriptions of the columns are available from [Kaggle](https://www.kaggle.com/c/house-prices-advanced-regression-techniques/data). For your convenience, you can review the descriptions of the data columns from [here](data-description.txt).

Pretend you are a data analyst at an investment company where the board decided to make investments in real estates. Your boss asked you to analyze this housing sales dataset and present to the investment managers on **what features of houses are strong indicators of the final sale price**. You need to present your findings in intuitive ways so that the investment managers understand where your conclusions come from.

#### You will use the appropriate data visualization graphs to tell your stories.

## Challenge 1 - Understanding the Dataset

After receiving the data and clarifying your objectives with your boss, you will first try to understand the dataset. This allows you to decide how you will start your research in the next step.

#### First, import the basic libraries and the dataset.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

df = pd.read_csv('housing_prices.csv')
pd.set_option('max_columns', 100)

#### As a routine before analyzing a dataset, print the first few rows of the dataset

In [ ]:
df.head()

You find the dataset has 81 columns which are a lot. 

#### Since the column `Id` is meaningless in our data visualization work, let's drop it

In [ ]:
df.drop(columns='Id',inplace=True)
df.head()

You care about missing values. If a column has too many missing values, it is not reliable to use it to predict sales price.

#### In the cell below, calculate the percentage of missing values for each column. 

Make a table containing the column name and the percentage of missing values. Print the columns where more than 20% of values are missing. An example of your output looks like:

![Missing Values](missing-values.png)

[This reference](https://stackoverflow.com/questions/51070985/find-out-the-percentage-of-missing-values-in-each-column-in-the-given-dataset) can help you make the missing values table.

In [ ]:
percent = 100 * df.isna().sum()/df.shape[0]
missing_val = pd.DataFrame(percent,columns=['percent_missing']).sort_values(by='percent_missing',ascending=False)
missing_val

#### Drop the columns you find that have more than 20% missing values.

After dropping, check the shape of your dataframes. You should have 75 columns now.

In [ ]:
df.drop(missing_val[missing_val['percent_missing']>20].index,axis=1,inplace=True)

In [ ]:
df.shape

Since you're asked to analyze sale prices, first let's see if the sale prices (column `SalePrice`) has a normal distribution. This is important because normally distributed data can be better represented with mathematical models.

#### In the cell below, use the propriate graph to visualize the shape of distribution of the sale prices. Then explain what you find from the graph about data distribution.

In [ ]:
sns.distplot(df.SalePrice);


In [ ]:
df.SalePrice.describe()

In [ ]:
"""
The distribution of SalePrice is slightly skewed to the left.
It is confirmed by the mean and the mediane that are not equals.

"""


## Bonus Challenge 1 - Adjust Data Distribution

If you used the correct method in the previous step, you should have found the data distribution is skewed to the left. In order to improve your data visualization in the next steps, you can opt to adjust the `SalePrice` column by applying a mathematical function to the values. The goal is to produce a bell-shape normal distribution after applying the mathematical function to the sale price.

*This technique is optional in data visualization but you'll find it useful in your future machine learning analysis.*

#### In the cell below, adjust the `SalePrice` column so that the data are normally distributed.

Try applying various mathematical functions such as square root, power, and log to the `SalePrice` column. Visualize the distribution of the adjusted data until you find a function that makes the data normally distributed. **Create a new column called `SalePriceAdjusted` to store the adjusted sale price.**

[This reference](https://trainingdatascience.com/workshops/histograms-and-skewed-data/) shows you examples on how to adjust skewed data.

In [ ]:
# Normalization to logarithmic 

df_exp = df.SalePrice.apply(np.log)
sns.distplot(df_exp);

df_exp.describe()

In [ ]:
# Normalization to Square Root 

df_sqrt = df.SalePrice.apply(np.sqrt)
sns.distplot(df_sqrt);

df_sqrt.describe()

In [ ]:
# Normalization to Power

df_pow = df.SalePrice.apply(lambda x: x**2)
sns.distplot(df_pow);

df_pow.describe()

In [ ]:
# Normalize with logarithmic function because it seems to be the most normally distributed data.

df['SalePriceAdjusted'] = df.SalePrice.apply(np.log)

## Challenge 2 - Exploring Data with Common Sense

Now that we have a general understanding of the dataset, we start exploring the data with common sense by means of data visualization. Yes, in data analysis and even machine learning you are often required to use common sense. You use your common sense to make a scientific guess (i.e. hypothesis) then use data analytics methods to test your hypothesis.

This dataset is about housing sales. According to common sense, housing prices depend on the following factors:

* **Size of the house** (`GrLivArea`, `LotArea`, and `GarageArea`).

* **Number of rooms** (`BedroomAbvGr`, `KitchenAbvGr`, `FullBath`, `HalfBath`, `BsmtFullBath`, `BsmtHalfBath`).

* **How long the house has been built or remodeled** (`YearBuilt` and `YearRemodAdd`).

* **Neighborhood of the house** (`Neighborhood`).

#### In this challenge, use the appropriate graph type to visualize the relationships between `SalePrice` (or `SalePriceAdjusted`) and the fields above. 

Note that:

* Transform certain columns in order to visualize the data properly based on common sense. For example:
    * Visualizing how the number of half bathrooms affected the sale price probably does not make sense. You can create a new column to calculate the total number of bathrooms/rooms then visualize with the calculated number.
    * `YearBuilt` and `YearRemodAdd` are year numbers not the age of the house. You can create two new columns for how long the house has been built or remodeled then visualize with the calculated columns.
* Make comments to explain your thinking process.

In [ ]:
# Create new columns to calculate how long the house has been built or remodeled

df['years_since_built'] = 2020-df.YearBuilt
df['years_since_remodeled'] = 2020-df.YearRemodAdd
df.iloc[:,-5:].head()

In [ ]:
# Creating a new column to get the number of bathroom per rooms above grade
df['HalfBath'] = df['HalfBath']/2
df['total_bathrooms'] = df[['FullBath', 'HalfBath']].sum(axis=1)
df['bath_per_rooms'] = df[['FullBath', 'HalfBath']].sum(axis=1) / df['TotRmsAbvGrd']
df.iloc[:,-5:].head()

In [ ]:
# Creating a new df filtered with the intesresting columns

sub_df = df[['GrLivArea', 'LotArea', 'GarageArea','BedroomAbvGr', 'KitchenAbvGr','Neighborhood']+list(df.iloc[:,-5:])]
sub_df.shape

In [ ]:
# Creating scatter plots to compare Sales Prices Adjusted with interesting features
fig, axes = plt.subplots(5,2, figsize=(10,17))

for idx,col in enumerate(sub_df.drop(columns='SalePriceAdjusted')):
    ax = axes[idx//2,idx%2]
    sns.scatterplot(data=sub_df,x=col,y='SalePriceAdjusted',ax=ax)


In [ ]:
"""
=> The Grade Living Area itself and the Garage Area seems to affect positively the price of houses,
compared to the Lot Area itself. 

We may assume that a house with a large basement area wouldn't make the price of the house 
higher even if it increases the total area of living. 

=> We can see the Neighborhood affects the price a lot. 

=> Having 1 from to 4 bedrooms above the grade affects positively the price, 
with 4 bedrooms which seems to be the sweet spot for an house with immediate higher price at the begining.

=> We may see a slight negative correlation between the years of building or remodeling and the price. 
That would mean the price may decrease as the years increase.

=> We can see the price is positively affected by the number of bathrooms above grade but 
it should stay relative to number of rooms above the grade.

"""

## Bonus Challenge 2 - Exploring Data with Correlation Heatmap

Now you have explored data visualizations with certain fields based on common sense. In the dataset there are many other fields that you are not sure whether they are important factors for the sale price. What is the best way to explore those fields without investigating them individually?

Making scatter matrix is not an option here because there are too many fields which makes it extremely time consuming to create scatter matrix. One option you have is to create a heatmap. Heatmaps are much less expensive to create than scatter matrixes. You can use heatmaps to visualize the pairwise correlations between each two variables.

Here is a [reference](https://seaborn.pydata.org/examples/many_pairwise_correlations.html) you can use to learn how to creat the pairwise correlation heatmap. Your heatmap should look like below:

![Corr Heatmap](heatmap.png)

In [ ]:
# Compute the correlation matrix
corr = df.corr()

# Generate a mask for the upper triangle
mask = np.triu(np.ones_like(corr, dtype=np.bool))

# Set up the matplotlib figure
f, ax = plt.subplots(figsize=(17, 12))

# Generate a custom diverging colormap
cmap = sns.diverging_palette(220, 10, as_cmap=True)

# Draw the heatmap with the mask and correct aspect ratio
sns.heatmap(corr, mask=mask, cmap=cmap, center=0,
            square=True, linewidths=.5, cbar_kws={"shrink": .5});


In your heatmap, you can easily identify the highly correlated (either positively or negatively) variables by looking for the grids with darker colors. 

#### In the cell below, summarize what variables are highly correlated to the sale price?

In [ ]:
"""
Most correlated variables to Sale Price is first of all the Over Quality (Overall material and finish quality),
which makes a lot of sense but it requires to know more about what kind of material and finish are qualitatives.

Otherwise, the grade living area, the garage area, if there is garage for cars, 
the total bathrooms and the total rooms above grade affects the price. 
Then, the neighborhood is also one feature that impact a lot the price.

Finally, we can see the age of the house (same for the age of remodeling) affect negatively the price. 

"""


## Challenge 3 - Present Your Stories

Now based on your findings from the explorations, summarize and present your stories.

#### Present the top 5 factors that affect the sale price.

Use the following format to present each factor:

1. A title line about the factor.

1. No more than 3 sentences to describe the relationship between the factor and the sale price.

1. Support your point with the appropriate graph.

## 1. Neighborhood

The neighborhood is one the main feature that affect the price. 
Depending on it, the start price for an house can immediately jump. 

The most expensive houses are located in the NoRidge neighborhood.

In [ ]:
plt.figure(figsize=(10,4))
sns.scatterplot(data=df,x='Neighborhood',y='SalePriceAdjusted')
plt.xticks(rotation=90)
plt.title('Sale Price by Neighborhood')
plt.show();

## 2. Overall Quality

The material and finish quality of the house affect a lot the price of house. The price clearly increase as the quality increase. 

The main point here would be to get to know what material and finish affect most the quality. 

In [ ]:
plt.figure(figsize=(10,4))
sns.regplot(data=df,x='OverallQual',y='SalePriceAdjusted')
plt.title('Correlation between Overall Quality and Sale Price',fontsize=14,pad=20)
plt.show();

## 3. Living Area

The Grade Living Area itself and the Garage Area seems to affect positively the price of houses,
compared to the Lot Area itself. 

Also, the price seems to increase as the more the garage can get cars. 

In [ ]:
fig, axes = plt.subplots(1,2, figsize=(17,4))

sns.scatterplot(data=df,x='GrLivArea',y='SalePriceAdjusted', color='b', ax=axes[0])
sns.scatterplot(data=df,x='GarageArea',y='SalePriceAdjusted', color='orange', ax=axes[1])
#sns.scatterplot(data=df,x='LotArea',y='SalePriceAdjusted', color='g', ax=axes[2])
fig.suptitle('Correlation between Area and Sale Price',fontsize=14)
plt.show()

## 4. Bathrooms but not too much

The price seems to increase when there is more bathrooms above the grade in the house. 

But we can see that the number of bathroom should be relative to the number of rooms in the house.
If there is too much bathrooms compared the the number of other rooms, the price won't be positively affected.

In [ ]:
fig, axes = plt.subplots(1,2, figsize=(14,4))

sns.scatterplot(data=df,x='total_bathrooms',y='SalePriceAdjusted',ax=axes[0])
sns.scatterplot(data=df,x='bath_per_rooms',y='SalePriceAdjusted',ax=axes[1])
fig.suptitle('Correlation between Bathrooms and Sale Price',fontsize=14)
plt.show()

## 5. Time matters

Becarful with how long the house has been built or remodeled. 

We can see that the price decrease slighly as the years of building or remodeling increase. 

In [ ]:
fig, axes = plt.subplots(1,2, figsize=(14,4))

sns.regplot(data=df,x='years_since_built',y='SalePriceAdjusted',ax=axes[0])
sns.regplot(data=df,x='years_since_remodeled',y='SalePriceAdjusted',ax=axes[1])
fig.suptitle('Correlation between Time and Sale Price',fontsize=14)
plt.show()